# App Data

> Class that loads and stores the data and sidebar for the dash app

In [ ]:
#| default_exp app_data

In [ ]:
# TODO think about what differentiates dash_components and app_data

In [ ]:
#| export
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
    from SolomonIslandsDataMap.dash_components import *
except: 
    from load_data import *
    from dash_components import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import numpy as np
from fastcore.test import *
from dash import Dash, dcc, Output, Input, State, html, Patch, ctx  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
import dash_mantine_components as dmc
import os

## Load Data

In [ ]:
#| export
# TODO data should be stored in store (or ) then loaded into pages.
sol_geo = SolomonGeo.load_pickle(aws = True)
geo_df = sol_geo.geo_df
fig = define_map(sol_geo)
# TODO save geo_df in store
#dcc.Store(id = 'selection',data = {})

## Setup Dash Components

Setup the app and variables to initialise the dropdowns

In [ ]:
geos = sol_geo.geo_levels
cen_vars = sol_geo.census_vars
NUM_GEOS = len(geos)

In [ ]:
# TODO up to here in this file, requires working function to create object from dict. Also the dict
# conversion doesn't work coz the index is not unique. Which needs to change. 
stored_data = dcc.Store(id='geo_df', data=geo_df.to_dict())

In [ ]:
type(geo_df.to_dict())

dict

Setup and create the dropdowns

In [ ]:
#| export
dropdown_location = html.Div(children = gen_dd(sol_geo.locations[sol_geo.geo_levels[0]], 
                                                'locDropdown', clear = True, place_holder='Select Dropdown Location',
                                                multi = True))

dropdown_geo = dmc.SegmentedControl(
                            id="segmented_geo",
                            value=geos[0],
                            data=geos,
                             orientation="vertical",
                            color = 'gray',
                            fullWidth = True,) # TODO consider redoing as theme is not consistent with this library
control_type = dmc.SegmentedControl(
                        id="segmented_type",
                        value=sol_geo.data_type[0],
                        data=sol_geo.data_type,
                        orientation="vertical",
                        color = 'gray',
                        fullWidth = True,)

dd_var = html.Div(children = gen_dd(list(sol_geo.census_vars.keys()), 'varDropdown', 
                                    val = 'Key Statistics', height = 75))
dd_measure = html.Div(children = gen_dd(sol_geo.census_vars['Key Statistics'], 'measureDropdown',
                                      val = sol_geo.census_vars['Key Statistics'][0]))

## Create a sidebar
The sidebar will 

In [ ]:
#| export
# Note, for now I am not using a sidebar style as I do not want to fix the width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "24rem",
    "padding": "2rem 1rem",
    #"background-color": "#f8f9fa",
}


sidebar = html.Div(
    [
        html.H2("Filters"),
        html.Hr(),
        dbc.Nav(
            [
                html.P("Geography"), # TODO add a tooltip button here with link to geo explanation
                dropdown_geo,
                html.Br(),
                html.P("Location"), # TODO add a little info button here with link to geo explanation
                dropdown_location,
                html.Br(),
                html.P("Data"), # TODO add a little info button here with link to geo explanation
                dd_var,
                dd_measure,
                html.Br(),
                html.P("Data Type"), 
                control_type,
                #html.Br(),
                #dcc.Dropdown(id = 'three')

            ],
            vertical=True,
            pills=True,
        ),
    ],
    #style=SIDEBAR_STYLE,
)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()